# Agent

# 準備

In [3]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv

# 環境変数の読み込み
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.environ["API_KEY"]

# Google検索Agent

In [1]:
from langchain.agents import Tool
from langchain_google_community import GoogleSearchAPIWrapper

search = GoogleSearchAPIWrapper()

tools = [
    Tool(
        name = "GoogleSearch", # 名前は半角英数字のみ
        func=search.run,
        description="Search Google for recent results."
    )
]

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=1.2)

In [5]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [6]:
from langchain.agents import create_openai_functions_agent

agent = create_openai_functions_agent(llm, tools, prompt)

In [7]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [9]:
agent_executor.invoke({"input": "IMF가 밝힌 대한민국의 2025년 경제 성장율은?"})



> Entering new AgentExecutor chain...

Invoking: `GoogleSearch` with `IMF South Korea GDP growth rate 2025`


15 indicator s , 1980 - 2029. Collapse all ... Real GDP growth ... Annual percent change ... 2.2 ... Created with Highcharts 6.1.4 ... 2025 ... GDP, current prices. Feb 5, 2025 ... Sources: Korean authorities; and IMF staff estimates and projections. 1/ Contribution to GDP growth. 2/ Excludes gold. ©International Monetary ... 2025 Projected Real GDP (% Change) : 2.0* · Country Population: 51.683 million · Date of Membership: August 26, 1955 · Article IV/Country Report: February 7, 2025 ... It has the 4th largest economy in Asia and the 12th largest in the world as of 2025. South Korea is notable for its rapid economic development from an ... Korea, Dem. People's Rep. of no data. Korea, Republic of 2.2. Kosovo 4. Kuwait 3.3. Kyrgyz Republic 5. Lao P.D.R. 3.5. Latvia 2.3. Lebanon no data. Lesotho 2.3. Oct 25, 2024 ... The International Monetary Fund (IMF) estimates South Korea'

{'input': 'IMF가 밝힌 대한민국의 2025년 경제 성장율은?',
 'output': 'IMF가 대한민국의 2025년 경제 성장률을 2.0%로 예상하고 있습니다.'}

# 対話型Google検索Agent

https://python.langchain.com/docs/modules/memory/agent_with_memory

In [10]:
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain_google_community import GoogleSearchAPIWrapper
from langchain_openai import ChatOpenAI

In [11]:
search = GoogleSearchAPIWrapper()
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="現在の出来事に関する質問に答える必要がある場合に便利です",
    )
]

In [12]:
prefix = """あなたは日本語で回答します。あなたは人間と会話し、次の質問にできる限り答えてください。次のツールにアクセスできます:"""

suffix = """開始！"

{chat_history}
Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "chat_history", "agent_scratchpad"],
)
memory = ConversationBufferMemory(memory_key="chat_history")

In [13]:
llm_chain = LLMChain(llm=ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=1.2), prompt=prompt)
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)
agent_chain = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True, memory=memory
)

d:\llm\sample_llm\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ZeroShotAgent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use create_react_agent instead.
  warn_deprecated(


In [14]:
agent_chain.invoke({"input":"日本の人口は何人ですか？"})



> Entering new AgentExecutor chain...
Thought: I should use the Search tool to find the current population of Japan
Action: Search
Action Input: "日本の人口 現在"
Observation: 各月1日現在の日本の人口について、最新の推計結果を掲載しています。 Jul 24, 2024 ... 【NHK】2024年1月1日現在の日本の総人口は1億2488万人余りで、前の年よりおよそ53万人減りました。日本人の人口が15年連… Apr 12, 2024 ... 総人口は1億2435万2千人で、前年に比べ59万5千人（-0.48％）の減少となり、13年連続で減少しています。 · 日本人人口は1億2119万3千人で、前年に比べ83万7千人 ... 日本の人口は、令和2年10月1日に行った 国勢 ( こくせい ) 調査 ( ちょうさ ) の結果によると、約1億2614万人です。いまから約400年前の江戸時代の初め（1600年ころ）は約 ... Apr 15, 2014 ... 総人口は1億2729万8千人で，前年に比べ21万7千人（0.17％）の減少と減少幅は縮小したものの，3年連続で大きく減少しています。 日本人人口は1億2570万4千人で， ... ※ 将来推計人口は、国立社会保障・人口問題研究所「日本の市区町村別将来推計人口（平成20年12月推計）」による。 ※ （ ）内の数値は、高齢化率。 ※ 大都市部は、埼玉県、千葉 ... 人口の推移、人口構造の変化. 日本の人口は近年減少局面を迎えており、2070年には総人口が9,000万人を割り込み、高齢化率は39％の水準になると推計されています。 · 平均寿命 ... Jan 1, 2024 ... 住民基本台帳に基づく人口、人口動態及び世帯数（令和6年1月1日現在） ... 【日本人住民】令和6年住民基本台帳人口・世帯数、令和5年人口動態（市区 ... d. 政治が良い。 ○だから、日本人は今も充分に健康。薬や医者に頼らず、. バランス良く食べ、ちゃんと寝て、体を使えばOK。 ○医師の人数が、経済先進国だと国民数百 ... 我が国の総人口は、令和5年10月1日現在、 ... 

{'input': '日本の人口は何人ですか？',
 'chat_history': '',
 'output': '現在の日本の人口は約1億2614万人です。'}

In [15]:
agent_chain.invoke(input="「その国の国歌を教えて")



> Entering new AgentExecutor chain...
Thought: I should search for the national anthem of Japan.
Action: Search
Action Input: National anthem of Japan
Observation: "Kimigayo" (君が代) is the national anthem of Japan. The lyrics are from a waka poem written by an unnamed author in the Heian period (794–1185), and the ... May 8, 2010 ... "Kimigayo" (君が代?) is Japan's national anthem. It is also one of the world's shortest national anthems in current use, with a length of 11 ... The Rising Sun Flag and "Kimi Ga Yo" are respectively the national flag and anthem of Japan. This was formalized in 1999 with the Law regarding the National ... Jul 26, 2023 ... 'Kimigayo' is the country's single-verse national anthem, based on the words of a classical Japanese waka poem written by an unknown author during Japan's ... Mar 20, 2025 ... For normal anthems he would put his hat over his heart like everyone else but for the Japanese anthems he would stand at attention, chest ... Nov 4, 2022 ... In this a

{'input': '「その国の国歌を教えて',
 'chat_history': 'Human: 日本の人口は何人ですか？\nAI: 現在の日本の人口は約1億2614万人です。',
 'output': '"Kimigayo" (君が代)'}

# Index検索Agent

In [16]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

# PDFの読込
loader = PyPDFLoader("./data2/001615363.pdf")
documents = loader.load()

# チャンクに分割
text_splitter = CharacterTextSplitter(separator="\n", chunk_size=512, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

# Indexの構築
db = Chroma.from_documents(texts, OpenAIEmbeddings())

# 検索（Retriever）の取得
retriever = db.as_retriever()

Created a chunk of size 636, which is longer than the specified 512


In [17]:
from langchain.tools.retriever import create_retriever_tool

tools = [
  create_retriever_tool(
      retriever,
      "ForeignTravelersQASystem",
      "訪日外国人の旅行や土産品、消費動向に関する質問に役立ちます。",
  )
]

In [18]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=1.2)

In [19]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [20]:
from langchain.agents import create_openai_functions_agent

agent = create_openai_functions_agent(llm, tools, prompt)

In [28]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [21]:
agent_executor.invoke({"input": "お土産の購入状況を教えて"})



> Entering new AgentExecutor chain...
お土産の購入状況に関する情報が必要ですね。お手伝いさせていただきます。購入されたお土産の種類や数、お買い物した場所など詳細情報を教えていただけますか？

> Finished chain.


{'input': 'お土産の購入状況を教えて',
 'output': 'お土産の購入状況に関する情報が必要ですね。お手伝いさせていただきます。購入されたお土産の種類や数、お買い物した場所など詳細情報を教えていただけますか？'}

# OpenAI Tool

In [22]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_openai import ChatOpenAI

In [23]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-tools-agent")

In [24]:
import requests
from bs4 import BeautifulSoup
from langchain.chains import TransformChain

# ニュースを検索する関数
def get_news(inputs: dict) -> dict:
    keyword = inputs["keyword"]

    # WebサイトのURLを指定
    url = f'https://news.google.com/search?q={keyword}&hl=ja&gl=JP&ceid=JP:ja'

    # Requestsを利用してWebページを取得する
    response = requests.get(url)
    html = response.text

    # BeautifulSoupを利用してWebページを解析する
    soup = BeautifulSoup(html, 'html.parser')

    # soup.selectを利用して、ニュースのタイトルを取得する
    elements = soup.select('article div div div a')

    # ニュースのトップ3を配列に取得
    news = []
    for i, element in enumerate(elements):
        news.append({"title": element.getText()})
        if i>=2:
            break

    return {"result": news}

# 関数をChain化
news_chain = TransformChain(
    input_variables=["keyword"],
    output_variables=["result"],
    transform=get_news
)


In [25]:
# 数式の文字列をもとに計算を行う関数
def calc_func(inputs: dict) -> dict:
    formula = inputs["formula"]
    answer = eval(formula)
    return {"answer": f"{answer}"}

# 関数をChain化
calc_chain = TransformChain(
    input_variables=["formula"],
    output_variables=["answer"],
    transform=calc_func
)

In [26]:
from langchain.agents import Tool
from langchain_google_community import GoogleSearchAPIWrapper

search = GoogleSearchAPIWrapper()

tools = [
    Tool(
        name = "news", # 半角英数字のみ
        func=news_chain.run,
        description="ニュースを探す際に役立ちます"
    ),
    Tool(
        name = "calculation", # 半角英数字のみ
        func=calc_chain.run,
        description="計算をする場合に役立ちます"
    ),
    Tool(
        name = "Search", # 半角英数字のみ
        func=search.run,
        description="現在の出来事や世界の現状に関する質問に答える必要がある場合に役立ちます"
    ),
]

In [27]:
# Choose the LLM that will drive the agent
# Only certain models support this
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

# Construct the OpenAI Tools agent
agent = create_openai_tools_agent(llm, tools, prompt)

In [28]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [29]:
agent_executor.invoke({"input": "LangChainとは何ですか?"})



> Entering new AgentExecutor chain...

Invoking: `Search` with `LangChain`


Build context-aware, reasoning applications with LangChain's flexible framework that leverages your company's data and APIs. LangChain helps developers build applications powered by LLMs through a standard interface for models, embeddings, vector stores, and more. Use LangChain for:. LangChain is a framework for developing applications powered by large language models (LLMs). LangChain simplifies every stage of the LLM application lifecycle. LangChain provides AI developers with tools to connect language models with external data sources. It is open-source and supported by an active community. New to LangChain or LLM app development in general? Read this material to quickly get up and running building your first applications. LangChain is a framework for developing applications powered by large language models (LLMs). LangChain simplifies every stage of the LLM application lifecycle. Dec 27, 2023 ... It is a

{'input': 'LangChainとは何ですか?',
 'output': 'LangChainは、大規模言語モデル（LLMs）によって動作するアプリケーションを開発するためのフレームワークです。LangChainは、LLMアプリケーションライフサイクルの各段階を簡素化します。LangChainは、AI開発者に言語モデルを外部データソースと接続するためのツールを提供します。LangChainはオープンソースであり、活発なコミュニティによってサポートされています。LangChainやLLMアプリケーション開発について初めての方は、この資料を読んで、すぐに最初のアプリケーションの構築を始めることができます。'}

In [30]:
agent_executor.invoke({"input": "8掛ける12は？"})



> Entering new AgentExecutor chain...

Invoking: `calculation` with `8*12`


968掛ける12は96です。

> Finished chain.


{'input': '8掛ける12は？', 'output': '8掛ける12は96です。'}

In [31]:
agent_executor.invoke({"input": "夏祭りに関するニュースを教えてください"})



> Entering new AgentExecutor chain...

Invoking: `news` with `夏祭り`


[{'title': '茅野市の夏祭り「どんばん」は8月9日開催に'}, {'title': '東方Project×よみうりランド 博麗神社夏祭り2025inよみうりランド'}, {'title': '今年も「超」が盛りだくさん！《ファイターズ 超夏祭り》開催'}]最新の夏祭りに関するニュースです：
1. 茅野市の夏祭り「どんばん」は8月9日開催に
2. 東方Project×よみうりランド 博麗神社夏祭り2025inよみうりランド
3. 今年も「超」が盛りだくさん！《ファイターズ 超夏祭り》開催

どれか特定のニュースについて詳細を知りたい場合はお知らせください。

> Finished chain.


{'input': '夏祭りに関するニュースを教えてください',
 'output': '最新の夏祭りに関するニュースです：\n1. 茅野市の夏祭り「どんばん」は8月9日開催に\n2. 東方Project×よみうりランド 博麗神社夏祭り2025inよみうりランド\n3. 今年も「超」が盛りだくさん！《ファイターズ 超夏祭り》開催\n\nどれか特定のニュースについて詳細を知りたい場合はお知らせください。'}

# Toolkit:Pandas DataFrame Agent

In [32]:
import pandas as pd

df = pd.read_csv("./data2/iris.data", header=0)
df.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Name
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [33]:
from langchain.agents.agent_types import AgentType
from langchain_openai import ChatOpenAI
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

In [34]:
agent = create_pandas_dataframe_agent(
    ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0125"),
    df,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
)

In [35]:
agent.invoke("何行ありますか？")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': 'df.shape[0]'}`


150データフレームには150行あります。

> Finished chain.


{'input': '何行ありますか？', 'output': 'データフレームには150行あります。'}

In [36]:
agent.invoke("Nameの基本統計量を出力してください")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df['Name'].describe()"}`


count             150
unique              3
top       Iris-setosa
freq               50
Name: Name, dtype: object`Name`列の基本統計量は以下の通りです:
- データ数（count）: 150
- ユニークな値の数（unique）: 3
- 最も頻繁に現れる値（top）: Iris-setosa
- 最も頻繁に現れる値の出現回数（freq）: 50

> Finished chain.


{'input': 'Nameの基本統計量を出力してください',
 'output': '`Name`列の基本統計量は以下の通りです:\n- データ数（count）: 150\n- ユニークな値の数（unique）: 3\n- 最も頻繁に現れる値（top）: Iris-setosa\n- 最も頻繁に現れる値の出現回数（freq）: 50'}

In [37]:
agent.invoke("このデータについて説明してください")



> Entering new AgentExecutor chain...
このデータは、アヤメの花（Iris）の品種ごとにセパル（がく片）とペタル（花弁）の長さと幅を示しています。各行は1つのアヤメの花を表し、列は以下の情報を提供しています：

- SepalLength: がく片の長さ
- SepalWidth: がく片の幅
- PetalLength: 花弁の長さ
- PetalWidth: 花弁の幅
- Name: アヤメの品種（このデータでは"Iris-setosa"のみ）

データの先頭から5行を示しており、それぞれの行が異なるアヤメの花を表しています。

> Finished chain.


{'input': 'このデータについて説明してください',
 'output': 'このデータは、アヤメの花（Iris）の品種ごとにセパル（がく片）とペタル（花弁）の長さと幅を示しています。各行は1つのアヤメの花を表し、列は以下の情報を提供しています：\n\n- SepalLength: がく片の長さ\n- SepalWidth: がく片の幅\n- PetalLength: 花弁の長さ\n- PetalWidth: 花弁の幅\n- Name: アヤメの品種（このデータでは"Iris-setosa"のみ）\n\nデータの先頭から5行を示しており、それぞれの行が異なるアヤメの花を表しています。'}

# Toolkit:Python Agent

In [38]:
from langchain import hub
from langchain.agents import AgentExecutor
from langchain_experimental.tools import PythonREPLTool

In [39]:
tools = [PythonREPLTool()]

In [40]:
from langchain.agents import create_openai_functions_agent
from langchain_openai import ChatOpenAI

In [41]:
instructions = """あなたは、質問に答えるために Python コードを作成して実行するように設計されたエージェントです。
Python コードを実行するために使用できる Python REPL にアクセスできます。
エラーが発生した場合は、コードをデバッグして再試行してください。
コードの出力は質問に答えるためにのみ使用してください。
コードを実行しなくても答えがわかる場合もありますが、答えを得るにはコードを実行する必要があります。
質問に答えるコードを書けそうにない場合は、「わかりません」と答えてください。
"""
base_prompt = hub.pull("langchain-ai/openai-functions-template")
prompt = base_prompt.partial(instructions=instructions)

In [42]:
agent = create_openai_functions_agent(ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0), tools, prompt)

In [43]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [44]:
agent_executor.invoke({"input": "100までの素数を求めよ"})



> Entering new AgentExecutor chain...


Python REPL can execute arbitrary code. Use with caution.



Invoking: `Python_REPL` with `def is_prime(n):
    if n <= 1:
        return False
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:
            return False
    return True

primes = [num for num in range(2, 101) if is_prime(num)]
primes`


100までの素数は、[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97] です。

> Finished chain.


{'input': '100までの素数を求めよ',
 'output': '100までの素数は、[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97] です。'}